In [1]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score, plot_confusion_matrix, balanced_accuracy_score

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import pandas as pd
from  sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.feature_selection import RFECV

training_path = 'data/data_train0.csv'
test_path = 'data/data_test.csv'


In [9]:
##Create Evaluation data frame to save the result of different experiments
evaluation_columns = ['Problem', 'Model', 'IsDatCorrect', 'Training_Accuracy', 'Testing_accuracy',
                      'Balanced_Training_Accuracy', 'Balanced_Testing_accuracy']

evaluation_data_frame= pd.DataFrame(columns=evaluation_columns)

def runLogisticModel(x,y):
    ## Prepare the model hyper parameters
    multi_class='auto'
    solver='liblinear'
    max_iter=1000
    class_weight='balanced'
    
    #Create the model
    LR = LogisticRegression(multi_class= multi_class, solver=solver, 
                            max_iter=max_iter, class_weight=class_weight)
    
    # Train the model
    LR = LR.fit(x, y)

    return LR

def runSvmModel(x,y):
   
    C=0.1
    kernel = 'rbf'
    gamma= 'auto'
    class_weight='balanced'
    decision_function_shape= 'ovr'
    tol = 1e-3
    
    #Create the model
    SVM = svm.SVC(C=C, kernel = kernel,
                  gamma= gamma, class_weight=class_weight, 
                  decision_function_shape= decision_function_shape, tol=tol)
    
    # Train the model
    SVM = SVM.fit(x,y)  
    
    return SVM

def runRandomForestModel(x,y):
    ## Prepare the model hyper parameters
    n_estimators=50
    max_leaf_nodes=20
    min_samples_split=0.01
    n_jobs=-1
    class_weight='balanced'
    
    RFC = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes,   
                                 min_samples_split=min_samples_split, n_jobs=n_jobs, 
                                 class_weight='balanced')
    # Train the model
    RFC =   RFC.fit(x,y)
    
    return RFC
    

def runDecisionTreeModel(x,y):
    #Prepare the model hyper parameters
    max_depth=15
    min_impurity_decrease=0.01
    #Create the model
    DTC = DecisionTreeClassifier(max_depth=max_depth, 
                                 min_impurity_decrease=min_impurity_decrease)
    
    #Train the model
    DTC = DTC.fit(x,y)
    
    return DTC


def runNNModel(x,y):
    #Prepare the model hyper parameters
    solver='adam'
    alpha=1e-7
    hidden_layer_sizes=(500,500,500)
    activation='relu'
    validation_fraction=0.2
    max_iter=100
    
    #Create the model and train it
    NN = MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes,
                       activation=activation, validation_fraction=validation_fraction, 
                       max_iter=max_iter)
    NN = NN.fit(x, y)
   
    return NN

def runExtraTreeModel(x,y):
    
    max_depth=10
    min_impurity_decrease=0.001
    class_weight='balanced' 
    
    ETM = ExtraTreeClassifier(max_depth=max_depth, 
                              min_impurity_decrease=min_impurity_decrease,  
                              class_weight=class_weight)
    
    ETM = ETM.fit(x,y)
    
    return ETM

def loadData(dataPath):
    data = pd.read_csv(dataPath)
    return data


def cleanData(data):
    cleanData = data.dropna(axis=0)
    return cleanData


def getColorData(data):
    x = data.loc[:,'lightness_0_0':'blueyellow_2_2']
    y = data.loc[:,['color']]
    #Encode the class and save the mapping dictionary to map it back later
    color_data_cat = y.astype('category')
    color_data_codes = color_data_cat['color'].cat.codes
    color_data_dict = color_data_cat['color'].cat.categories
  
    return x, color_data_codes, color_data_dict

def getTextureData(data):
    x = data.loc[:,'hog_0_0_0':]
    y = data.loc[:,['texture']]
    texture_data_cat = y.astype('category')
    texture_data_codes = texture_data_cat['texture'].cat.codes
    texture_data_dict = texture_data_cat['texture'].cat.categories
  
    return x, texture_data_codes, texture_data_dict

def mapData(data, dictionary, column):
    ## Map the classes back after prediction
    mappedData = data[column].map(di)
    
    return mappedData
    

def showConfusionMatrix(model, x_train, y_train, x_test, y_test):
    
    return

evaluation_columns = ['Problem', 'Model', 'IsDatCorrect', 'Training_Accuracy', 'Testing_accuracy',
                      'Balanced_Training_Accuracy', 'Balanced_Testing_accuracy']

evaluation_data_frame = pd.DataFrame(columns=evaluation_columns)

def evaluatePerformance(model, x_train, x_test, y_train, y_test, problem, modelName, modelCorrect):
    global evaluation_data_frame
    
    ## Get training accuracy and testing accuracy, balanced and unbalanced
    training_pred = model.predict(x_train)
    testing_pred = model.predict(x_test)
    
    training_accuracy = accuracy_score(training_pred, y_train)
    training_balanced_accuracy = balanced_accuracy_score(training_pred, y_train)
    testing_accuracy = accuracy_score(testing_pred, y_test)
    testing_balanced_accuracy = balanced_accuracy_score(testing_pred, y_test)
    
    ## Create a record to add to the evaulation data frame
    result_record = [problem, modelName, modelCorrect, training_accuracy, testing_accuracy, 
                    training_balanced_accuracy, testing_balanced_accuracy]
    series = pd.Series(result_record, index = evaluation_columns)
    
    ## Append the record to the datafame to conduct data analysis later
    evaluation_data_frame = evaluation_data_frame.append(series, ignore_index=True)
    


def visalizeColor(data):
    
    col_names = ['lightness', 'redgreen','blueyellow']
    #Create a new data frame
    x = data.loc[:,'lightness_0_0':'blueyellow_2_2']
    new_data = pd.DataFrame(columns=col_names)
    
    #split each area of the 9 areas to a different class taking the same color of the main object
    for i in range(3):
        for j in range(3):
            temp_df =  pd.DataFrame()
            color_index = '_' + str(i) + '_' + str(j)
            for col in col_names:
                temp_df[str(col)] = x[col + color_index]
            temp_df['color'] = color_data
            new_data = new_data.append(temp_df)

    #Prpeare a  3D plot
    plt.rcParams["figure.figsize"] = (20,20)

    fig = plt.figure()

    ax = fig.add_subplot(111, projection='3d')
    #Plot the data and set labels
    ax.scatter(new_data['lightness'], new_data['redgreen'],  new_data['blueyellow'], c=new_data["color"])
    ax.set_xlabel('Red green')
    ax.set_ylabel('Blue yellow')
    ax.set_zlabel('Lightness')
    ax.set_title("Color distribution:")
    plt.show()


In [10]:
#Load data and clean it
training_data = loadData(training_path)
clean_data = cleanData(training_data)

## separate color and texture data
x_color, y_color, color_dictionary = getColorData(clean_data)
x_texture, y_texture, texture_dictionary = getTextureData(clean_data)

## Split the data to training and validation

x_color_train, x_color_validate, y_color_train, y_color_validate = train_test_split(x_color, y_color,  test_size=0.2)
x_texture_train, x_texture_validate, y_texture_train, y_texture_validate = train_test_split(x_texture, 
                                                                                            y_texture,  
                                                                                            test_size=0.2)
## Scale the continuous features
x_color_train, x_color_validate, x_texture_train, x_texture_validate = (scale(x_color_train), 
                                                                        scale(x_color_validate), 
                                                                        scale(x_texture_train), 
                                                                        scale(x_texture_validate))


In [11]:
## Expirment 1:
##Train correctly 5 models on color and 5 on texture
## and save the evaluation result ot the evaluation dataframe to analyse data later

## Extra Tree Model
model_correctness = "Correct Data"
color_extra_tree_model = runExtraTreeModel(x_color_train, y_color_train)
evaluatePerformance(color_extra_tree_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Extra Tree", model_correctness)

texture_extra_tree_model = runExtraTreeModel( x_texture_train, y_texture_train)
evaluatePerformance(texture_extra_tree_model, x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Extra Tree", model_correctness)

## Decision supportVecotr Model
color_svm_model = runSvmModel(x_color_train, y_color_train)
evaluatePerformance(color_svm_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "SVM", model_correctness)

texture_svm_model = runSvmModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_svm_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "SVM", model_correctness)

#Logistic Regression Models
color_logistic_model = runLogisticModel(x_color_train, y_color_train)
evaluatePerformance(color_logistic_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Logistic Regression", model_correctness)

texture_logistic_model = runLogisticModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_logistic_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Logistic Regression", model_correctness)

## Random FOrest Model
color_random_forest_model = runRandomForestModel(x_color_train, y_color_train)
evaluatePerformance(color_random_forest_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Random Forest", model_correctness)

texture_random_forest_model = runRandomForestModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_random_forest_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Random Forest", model_correctness)


## Decision Tree Model
color_decision_tree_model = runDecisionTreeModel(x_color_train, y_color_train)
evaluatePerformance(color_decision_tree_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Decision Tree", model_correctness)
texture_decision_tree_model = runDecisionTreeModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_decision_tree_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Decision Tree", model_correctness)

## Neural netwrok Model
color_neural_network_model = runNNModel(x_color_train, y_color_train)
evaluatePerformance(color_neural_network_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Neural Network", model_correctness)

texture_neural_network_model = runNNModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_neural_network_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Neural Network", model_correctness)




C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packag

In [12]:
## Expirment 2:
##Train Wrong 5 models on color and 5 on texture

## and save the evaluation result ot the evaluation dataframe to analyse data later

## Make a wrong data by given the HOGs and BIMP features to predict the texture 
## and the texture features to predict the color

x_color_train, x_color_validate, y_color_train, y_color_validate = train_test_split(x_texture, y_color,  test_size=0.2)
x_texture_train, x_texture_validate, y_texture_train, y_texture_validate = train_test_split(x_color , y_texture,  test_size=0.2)

## Scale the data
x_color_train, x_color_validate, x_texture_train, x_texture_validate = (scale(x_color_train), 
                                                                        scale(x_color_validate), 
                                                                        scale(x_texture_train), 
                                                                        scale(x_texture_validate))
## Extra Tree Model
model_correctness = "Wrong Data"
color_extra_tree_model = runExtraTreeModel(x_color_train, y_color_train)
evaluatePerformance(color_extra_tree_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Extra Tree", model_correctness)

texture_extra_tree_model = runExtraTreeModel( x_texture_train, y_texture_train)
evaluatePerformance(texture_extra_tree_model, x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Extra Tree", model_correctness)

## Decision supportVecotr Model
color_svm_model = runSvmModel(x_color_train, y_color_train)
evaluatePerformance(color_svm_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "SVM", model_correctness)

texture_svm_model = runSvmModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_svm_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "SVM", model_correctness)

#Logistic Regression Models
color_logistic_model = runLogisticModel(x_color_train, y_color_train)
evaluatePerformance(color_logistic_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Logistic Regression", model_correctness)

texture_logistic_model = runLogisticModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_logistic_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Logistic Regression", model_correctness)

## Random FOrest Model
color_random_forest_model = runRandomForestModel(x_color_train, y_color_train)
evaluatePerformance(color_random_forest_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Random Forest", model_correctness)

texture_random_forest_model = runRandomForestModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_random_forest_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Random Forest", model_correctness)


## Decision Tree Model
color_decision_tree_model = runDecisionTreeModel(x_color_train, y_color_train)
evaluatePerformance(color_decision_tree_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Decision Tree", model_correctness)
texture_decision_tree_model = runDecisionTreeModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_decision_tree_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Decision Tree", model_correctness)

## Neural netwrok Model
color_neural_network_model = runNNModel(x_color_train, y_color_train)
evaluatePerformance(color_neural_network_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Neural Network", model_correctness)

texture_neural_network_model = runNNModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_neural_network_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Neural Network", model_correctness)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
C:\ProgramData\Anaconda3\lib\site-packag

In [ ]:
## Expirment 3:
##Train The models on completely random data

## and save the evaluation result ot the evaluation dataframe to analyse data later

## Make a wrong data by given the HOGs and BIMP features to predict the texture 
## and the texture features to predict the color

## Create Random Data

x_texture = np.random

x_color_train, x_color_validate, y_color_train, y_color_validate = train_test_split(x_texture, y_color,  test_size=0.2)
x_texture_train, x_texture_validate, y_texture_train, y_texture_validate = train_test_split(x_color , y_texture,  test_size=0.2)

## Extra Tree Model
model_correctness = "Random Data"
color_extra_tree_model = runExtraTreeModel(x_color_train, y_color_train)
evaluatePerformance(color_extra_tree_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Extra Tree", model_correctness)

texture_extra_tree_model = runExtraTreeModel( x_texture_train, y_texture_train)
evaluatePerformance(texture_extra_tree_model, x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Extra Tree", model_correctness)

## Decision supportVecotr Model
color_svm_model = runSvmModel(x_color_train, y_color_train)
evaluatePerformance(color_svm_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "SVM", model_correctness)

texture_svm_model = runSvmModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_svm_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "SVM", model_correctness)

#Logistic Regression Models
color_logistic_model = runLogisticModel(x_color_train, y_color_train)
evaluatePerformance(color_logistic_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Logistic Regression", model_correctness)

texture_logistic_model = runLogisticModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_logistic_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Logistic Regression", model_correctness)

## Random FOrest Model
color_random_forest_model = runRandomForestModel(x_color_train, y_color_train)
evaluatePerformance(color_random_forest_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Random Forest", model_correctness)

texture_random_forest_model = runRandomForestModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_random_forest_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Random Forest", model_correctness)


## Decision Tree Model
color_decision_tree_model = runDecisionTreeModel(x_color_train, y_color_train)
evaluatePerformance(color_decision_tree_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Decision Tree", model_correctness)
texture_decision_tree_model = runDecisionTreeModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_decision_tree_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Decision Tree", model_correctness)

## Neural netwrok Model
color_neural_network_model = runNNModel(x_color_train, y_color_train)
evaluatePerformance(color_neural_network_model, x_color_train, x_color_validate, 
                    y_color_train, y_color_validate,"Color", "Neural Network", model_correctness)

texture_neural_network_model = runNNModel(x_texture_train, y_texture_train)
evaluatePerformance(texture_neural_network_model,x_texture_train, x_texture_validate, y_texture_train,
                    y_texture_validate,"Texture", "Neural Network", model_correctness)


In [13]:
print(evaluation_data_frame)

    Problem                Model IsDatCorrect  Training_Accuracy  \
0     Color           Extra Tree         True           0.425492   
1   Texture           Extra Tree         True           0.402999   
2     Color                  SVM         True           0.298969   
3   Texture                  SVM         True           0.301781   
4     Color  Logistic Regression         True           0.335520   
5   Texture  Logistic Regression         True           0.931584   
6     Color        Random Forest         True           0.419869   
7   Texture        Random Forest         True           0.572634   
8     Color        Decision Tree         True           0.424555   
9   Texture        Decision Tree         True           0.465792   
10    Color       Neural Network         True           0.976570   
11  Texture       Neural Network         True           1.000000   
12    Color           Extra Tree        False           0.417057   
13  Texture           Extra Tree        False   

In [14]:
evaluation_data_frame.to_csv("Model Testing results.csv")

In [ ]:

clf = RandomForestClassifier(n_estimators=12, max_leaf_nodes=10,   min_samples_split=0.01,  n_jobs=-1 )
sel = RFECV(estimator=clf, step=2, cv=50)
sel = sel.fit(x1_train,color_train)
sel.support_
#clf.fit(x1_train,color_train)
#pred = clf.predict(x1_validate)
#p1 = clf.predict(x1_train)
#p2 = clf.predict(x1_validate)
#show_acc(p1,color_train.values)
#show_acc(p2,color_validate.values)

In [ ]:
print(sel.get_support())

In [ ]:
print(len(sel.get_support()))